##### Deep Learning to Solve Higher Differential Equations

In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
     raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
print(tf.__version__)

Found GPU at: /device:GPU:0
2.17.0


In [11]:
import time
import math
import tensorflow.compat.v1 as tf
import numpy as np
from tensorflow.python.training.moving_averages import assign_moving_average  # Moving average
from scipy.stats import multivariate_normal as normal  # Generate normally distributed random numbers
from tensorflow.python.ops import control_flow_ops  # Used for control flow
from tensorflow import random_normal_initializer as norm_init  # Tensor initializer with normal distribution
from tensorflow import random_uniform_initializer as unif_init  # Tensor initializer with uniform distribution
from tensorflow import constant_initializer as const_init  # Tensor initializer with constant values
from tensorflow.keras.layers import LSTMCell

tf.disable_v2_behavior()

class SolveAllenCahn(object):
    """ The fully-connected neural network model """
    def __init__(self, sess):
        self.sess = sess  # Session
        # parameters for the PDE
        self.d = 100  # Dimension of the data
        self.T = 0.3  # Time length for each path
        # parameters for the algorithm
        self.n_time = 20  # Number of networks
        self.batch_size = 64  # Use paths in batches for calculations, 64*4=256
        self.valid_size = 256  # 256 Monte Carlo samples (paths)
        self.n_maxstep = 4000  # Iteration steps
        self.n_displaystep = 100  # Print every 100 steps
        self.learning_rate = 5e-4  # Learning rate
        self.Yini = [0.3,0.6]  # Min and max values for initial Y0
        # some basic constants and variables
        self.h = (self.T + 0.0) / self.n_time  # Data time interval for each path, delta t
        self.sqrth = math.sqrt(self.h)  # sqrt(delta t), used later in calculations
        self.t_stamp = np.arange(0, self.n_time) * self.h  # Time stamps, cumulative time
        self._extra_train_ops = []  # Batch moving average operation, includes additional trainable beta and gamma

    def train(self):
        # Main function for neural network training
        start_time = time.time()  # Start time
        # Create new TensorFlow variable, name='global_step', as a generator for consistency
        self.global_step = tf.get_variable('global_step', [], initializer=tf.constant_initializer(1),
                                           trainable=False, dtype=tf.int32)  # Not added to the trainable variable list, just a counter
        trainable_vars = tf.trainable_variables()  # View trainable variables
        grads = tf.gradients(self.loss, trainable_vars)  # Gradients for loss trainable variables
        optimizer = tf.train.AdamOptimizer(self.learning_rate)  # Gradient optimizer
        apply_op = optimizer.apply_gradients(zip(grads, trainable_vars),  # Apply gradients to update trainable_vars list
                                             global_step=self.global_step)  # Update gradients and iteration count

        train_ops = [apply_op] + self._extra_train_ops  # Add operations, similar to list1.extend(list2)
        self.train_op = tf.group(*train_ops)  # tf.group(*train_ops) combines operations in *train_ops

        self.loss_history = []  # To record loss values
        self.init_history = []  # To record Y0 values
        self.run_time = []  # To record runtime

        # For validation, 256 Monte Carlo samples as the validation set
        dW_valid, X_valid = self.sample_path(self.valid_size)  # Generate data
        feed_dict_valid = {self.dW: dW_valid,  # Feed data to placeholders in buildmodel
                           self.X: X_valid,
                           self.is_training: False}  # Exclude from iteration
        # Initialization
        step = 1
        self.sess.run(tf.global_variables_initializer())  # Initialize global variables

        # Run framework
        temp_loss = self.sess.run(self.loss, feed_dict=feed_dict_valid)  # Calculate loss

        temp_init = self.Y0.eval()[0]  # Extract values, Y0 is a 2D tensor
        self.loss_history.append(temp_loss)  # Record loss
        self.init_history.append(temp_init)  # Record Y0
        self.run_time.append(time.time() - start_time + self.t_bd)
        print("step : %5u , loss : %.4e , " % \
              (0, temp_loss) + \
              "Y0 : % .4e , runtime : %4u " % \
              (temp_init, time.time() - start_time + self.t_bd))  # Print state for step=0

        # Start SGD iteration, steps 0-4000
        for _ in range(self.n_maxstep + 1):
            step = self.sess.run(self.global_step)
            dW_train, X_train = self.sample_path(self.batch_size)  # Generate data
            self.sess.run(self.train_op,
                          feed_dict={self.dW: dW_train,  # Feed data to placeholders in buildmodel
                                     self.X: X_train,
                                     self.is_training: True})
            if step % self.n_displaystep == 0:  # Test with validation set every 100 steps for loss and Y0
                temp_loss = self.sess.run(self.loss, feed_dict=feed_dict_valid)
                temp_init = self.Y0.eval()[0]  # Extract values
                self.loss_history.append(temp_loss)  # Loss values
                self.init_history.append(temp_init)  # Y0 values, Y0 is a 2D tensor
                self.run_time.append(time.time() - start_time + self.t_bd)
                print("step : % 5u , loss : %.4e , " % \
                      (step, temp_loss) + \
                      " Y0 : % .4e , runtime : %4u " % \
                      (temp_init, time.time() - start_time + self.t_bd))
            step += 1
        end_time = time.time()  # End time for training
        print(" running time : % .3f s " % \
              (end_time - start_time + self.t_bd))

    def build(self):
        # Build the whole network by stacking subnetworks
        start_time = time.time()
        # Placeholders for dW, X, and is_training, batch size is None to process one batch at a time
        self.dW = tf.placeholder(tf.float32, [None, self.d, self.n_time], name='dW')  # None*100*20
        self.X = tf.placeholder(tf.float32, [None, self.d, self.n_time + 1], name='X')  # None*100*20
        self.is_training = tf.placeholder(tf.bool)

        # Initialize Y0 and Z0
        self.Y0 = tf.Variable(tf.random_uniform([1],  # Initial u0, one value for one dimension
                                                minval=self.Yini[0],  # Min value 0.3
                                                maxval=self.Yini[1],  # Max value 0.6
                                                dtype=tf.float32))
        self.Z0 = tf.Variable(tf.random_uniform([1, self.d],  # Initial value for gradient u, a 1*d vector
                                                minval=-.1,  # Min value
                                                maxval=.1,  # Max value
                                                dtype=tf.float32))
        self.allones = tf.ones(shape=tf.stack([tf.shape(self.dW)[0], 1]),  # tf.shape(self.dW)[0]=len(batch), shape=(batch,1)
                               dtype=tf.float32)  # Batch generation of initial values

        Y = self.allones * self.Y0  # Initial Y as input, each batch gets the same initial Y value, Y is a (batch,1) 2D matrix [[],[],..,]
        Z = tf.matmul(self.allones, self.Z0)  # Initial Z as output, similar to Y, but as Z is a vector, it’s a (batch, d) matrix

        with tf.variable_scope('forward', reuse=tf.AUTO_REUSE):  # Forward propagation
            cell = LSTMCell(units=110)
            batch_size = tf.shape(self.X)[0]
            hid = [tf.zeros([batch_size, cell.units]), tf.zeros([batch_size, cell.units])]
            w = tf.get_variable('Matrixhid',
                                [110, 100], tf.float32,
                                norm_init(stddev=5 / np.sqrt(110)))
            for t in range(0, self.n_time - 1):  # Network for the first N-1 xt
                # Computation from recursive formula
                Y = Y - self.f_tf(self.t_stamp[t],  # Timestamp, cumulative time value
                                  self.X[:, :, t], Y, Z) * self.h  # Y.shape=(batch,1)
                Y = Y + tf.reduce_sum(Z * self.dW[:, :, t], 1, keep_dims=True)  # Intermediate time output.
                output, hid = cell(self._batch_norm(self.X[:, :, t + 1], 'normal'), hid)  # Update hid at each time
                Z = tf.nn.relu(tf.matmul(output, w)) / self.d

            # Terminal time, as final Y does not use neural network
            Y = Y - self.f_tf(self.t_stamp[self.n_time - 1],  # -1 because index starts from 0, terminal step is different
                              self.X[:, :, self.n_time - 1], Y, Z) * self.h
            Y = Y + tf.reduce_sum(Z * self.dW[:, :, self.n_time - 1], 1, keep_dims=True)  # Final Y output
            term_delta = Y - self.g_tf(self.T, self.X[:, :, self.n_time])  # Loss function
            self.clipped_delta = tf.clip_by_value(term_delta, -50.0, 50.0)  # Clip values within bounds
            self.loss = tf.reduce_mean(self.clipped_delta ** 2)  # Calculate loss
        self.t_bd = time.time() - start_time  # Time to generate the network

    def sample_path(self, n_sample):
        # Generate paths, creating (xt, (wt-wt-1))
        dW_sample = np.zeros([n_sample, self.d, self.n_time])  # Sample count, dimension, time length
        X_sample = np.zeros([n_sample, self.d, self.n_time + 1])
        for i in range(self.n_time):  # Generate one column at a time
            dW_sample[:, :, i] = np.reshape(normal.rvs(mean=np.zeros(self.d),  # This function is similar to np.random.normal()
                                                       cov=1,  # Why not std=1 ?
                                                       size=n_sample) * self.sqrth,  # sqrt(delta t), W(t)-W(s) is independent, with mean 0 and variance t-s
                                              (n_sample, self.d))
            X_sample[:, :, i + 1] = X_sample[:, :, i] + np.sqrt(2) * dW_sample[:, :, i]  # From formula
        return dW_sample, X_sample

    def f_tf(self, t, X, Y, Z):
        # Nonlinear term
        return Y - tf.pow(Y, 3)

    def g_tf(self, t, X):
        # Terminal conditions
        return 0.5 / (1 + 0.2 * tf.reduce_sum(X ** 2, 1, keep_dims=True))

    def _batch_norm(self, x, name):
        """ Batch normalization """  # Beta and gamma are trainable, third type of parameter, needs 2 columns for each normalization
        with tf.variable_scope(name):
            params_shape = [x.get_shape()[-1]]  # [d, d+10, d+10, d], first dimension is batch
            beta = tf.get_variable('beta', params_shape, tf.float32, norm_init(0.0, stddev=0.1))
            gamma = tf.get_variable('gamma', params_shape, tf.float32, unif_init(0.1, 0.5))
            mv_mean = tf.get_variable('moving_mean',  # Moving_mean improves mean for different batches
                                      params_shape, tf.float32, const_init(0.0), trainable=False)
            mv_var = tf.get_variable('moving_variance',
                                     params_shape, tf.float32, const_init(1.0), trainable=False)

            # These ops will only be performed when training
            mean, variance = tf.nn.moments(x, [0], name='moments')  # Centered dimension for normalization, [0] means batch, calculate mean and variance for 64 samples
            self._extra_train_ops.append(assign_moving_average(mv_mean, mean, 0.99))  # Explained below
            self._extra_train_ops.append(assign_moving_average(mv_var, variance, 0.99))

            mean, variance = tf.cond(self.is_training,  # control_flow_ops.cond controls execution flow, first parameter is the condition
                                                   lambda: (mean, variance),  # If True, calculate mean and variance
                                                   lambda: (mv_mean, mv_var))  # If False, directly use smoothed value

            y = tf.nn.batch_normalization(x, mean, variance, beta, gamma, 1e-6)
            # Above operation is equivalent to:
            # y = (y - mean) / tf.sqrt(variance+1e-6)  # 1e-6 epsilon
            # y = y * gamma + beta

            # Ensure that shape is maintained after normalization
            y.set_shape(x.get_shape())
            return y


def main(name):
    tf.reset_default_graph()
    with tf.Session() as sess:
        tf.set_random_seed(1)  # Random seed in tf
        print("Begin to solve Allen - Cahn equation")
        model = SolveAllenCahn(sess)  # Create object
        model.build()  # Call object method, constructs a model but does not feed data
        print('End build')
        model.train()  # Generate and feed data into build
        output = np.zeros((len(model.init_history), 4))  # Initialize result as 0, fill later
        output[:, 0] = np.arange(len(model.init_history)) * model.n_displaystep  # Output step
        output[:, 1] = model.loss_history  # Output loss list
        output[:, 2] = model.init_history  # Output Y0 list
        output[:, 3] = model.run_time
        np.savetxt("./AllenCahn_d100_RNN" + str(name) + ".csv",  # Save output results
                   output,
                   fmt=['%d', '%.5e', '%.5e', '%d'],
                   delimiter=",",
                   header="step ,loss function , target value , runtime ",
                   comments='')


if __name__ == '__main__':
    np.random.seed(1)  # Define a random seed
    for i in range(5):
        print(str(i) + ' run:')
        main(i)  # Run main program


0 run:
Begin to solve Allen - Cahn equation
End build
step :     0 , loss : 1.6388e-01 , Y0 :  5.4381e-01 , runtime :    7 
step :   100 , loss : 1.2872e-01 ,  Y0 :  4.9521e-01 , runtime :   31 
step :   200 , loss : 1.0083e-01 ,  Y0 :  4.5013e-01 , runtime :   51 
step :   300 , loss : 7.9346e-02 ,  Y0 :  4.0851e-01 , runtime :   73 
step :   400 , loss : 6.2092e-02 ,  Y0 :  3.6999e-01 , runtime :   92 
step :   500 , loss : 4.8334e-02 ,  Y0 :  3.3424e-01 , runtime :  114 
step :   600 , loss : 3.6952e-02 ,  Y0 :  3.0123e-01 , runtime :  134 
step :   700 , loss : 2.8076e-02 ,  Y0 :  2.7067e-01 , runtime :  156 
step :   800 , loss : 2.0854e-02 ,  Y0 :  2.4251e-01 , runtime :  178 
step :   900 , loss : 1.5507e-02 ,  Y0 :  2.1678e-01 , runtime :  197 
step :  1000 , loss : 1.1445e-02 ,  Y0 :  1.9347e-01 , runtime :  216 
step :  1100 , loss : 8.2324e-03 ,  Y0 :  1.7236e-01 , runtime :  241 
step :  1200 , loss : 5.8491e-03 ,  Y0 :  1.5355e-01 , runtime :  264 
step :  1300 , loss : 4.